In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=20):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=20):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [4]:
#read in chicago validation labels
chicago_labels = gpd.read_file('data/round-1/chicago-validation-labels-round1-230611.geojson')
chicago_labels = chicago_labels.drop(columns=['time_created'])
chicago_labels.head()

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,94,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,t,4.0,f,"not enough landing space,narrow,not level with...",None,POINT (-87.72816 42.02261)
1,96,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,t,3.0,f,"not enough landing space,surface problem",None,POINT (-87.72815 42.02272)
2,97,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,t,3.0,f,"not enough landing space,points into traffic",None,POINT (-87.72843 42.02272)
3,98,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,t,3.0,f,not enough landing space,None,POINT (-87.72844 42.02262)
4,99,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,t,1.0,f,grass,None,POINT (-87.72815 42.02300)


In [5]:
#read in chicago streets
chicago_streets = gpd.read_file("data/round-1/chicago-streets-230611.geojson")

In [6]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to chicago_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [7]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [10]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [13]:
alpha_chicago = {}
for i in [0,1,2,4,5,8,9]:
    alpha = krippendorff_alpha(chicago_labels,i, chicago_streets, 3435)
    alpha_chicago[label_types[i]] = alpha

In [14]:
#30 ft segments
alpha_chicago

{'CurbRamp': 0.927,
 'NoSidewalk': 0.663,
 'Problem': 0.565,
 'SurfaceProblem': 0.53,
 'Obstacle': 0.506,
 'Crosswalk': 0.946,
 'Signal': 0.818}

surface problem and obstacle without severity level =1

In [15]:
#take out label type = surfaceproblem and obstacle
chicago_problems= chicago_labels[(chicago_labels['label_type'] == 'SurfaceProblem') | (chicago_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
chicago_problems = chicago_problems[chicago_problems['severity'] != 1]
chicago_problems

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
15,113,Devon Snyder,113,437,Skokie,Obstacle,42.024498,-87.728394,p_-iwPEGdz3zFZzojysq_Q,t,4.0,t,construction,None,POINT (-87.72839 42.02450)
16,114,Devon Snyder,114,447,Skokie,Obstacle,42.024517,-87.728065,p_-iwPEGdz3zFZzojysq_Q,t,5.0,t,construction,None,POINT (-87.72807 42.02452)
22,120,Devon Snyder,120,447,Skokie,SurfaceProblem,42.025555,-87.728050,6-jUvCDerpU1Mmdck_7brw,t,2.0,f,cracks,None,POINT (-87.72805 42.02555)
23,121,Devon Snyder,121,447,Skokie,SurfaceProblem,42.025524,-87.728058,6-jUvCDerpU1Mmdck_7brw,t,3.0,f,"cracks,uneven/slanted",None,POINT (-87.72806 42.02552)
24,122,Devon Snyder,122,447,Skokie,SurfaceProblem,42.025539,-87.728050,6-jUvCDerpU1Mmdck_7brw,t,3.0,f,uneven/slanted,None,POINT (-87.72805 42.02554)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,2116,mikey,2116,417,Robbins,SurfaceProblem,41.641541,-87.699074,pyGMeg_ov_yzl9DHeCNKaQ,None,3.0,f,grass,None,POINT (-87.69907 41.64154)
462,2130,mikey,2130,445,Robbins,SurfaceProblem,41.644279,-87.699326,aezvzNgq2vdXD_QT4edWRw,None,2.0,f,None,None,POINT (-87.69933 41.64428)
471,2139,mikey,2139,445,Robbins,SurfaceProblem,41.644253,-87.700073,LyeTlktgRkxWxkPeFoZ63w,None,2.0,f,"grass,uneven/slanted",None,POINT (-87.70007 41.64425)
476,2144,mikey,2144,445,Robbins,Obstacle,41.644253,-87.700371,Hdhuc9wypqqkvSIEWjpx3w,None,4.0,f,trash/recycling can,None,POINT (-87.70037 41.64425)


In [16]:
label_types

['CurbRamp',
 'NoSidewalk',
 'Problem',
 'Occlusion',
 'SurfaceProblem',
 'Obstacle',
 'Other',
 'NoCurbRamp',
 'Crosswalk',
 'Signal']

In [17]:
alpha_chicago_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(chicago_problems,i, chicago_streets, 3435)
    alpha_chicago_problems[label_types[i]] = alpha

In [18]:
alpha_chicago_problems

{'Problem': 0.48, 'SurfaceProblem': 0.468, 'Obstacle': 0.578}

In [15]:
# #plot severity counts for label type = surfaceproblem and obstacle
# sns.countplot(x='severity', hue='label_type', data=chicago_problems)
# #add title
# plt.title('Severity Counts for Surface Problem and Obstacle Labels Chicago')


In [16]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(chicago_labels,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_" + label_types[i]+ "_230608.html")



In [17]:
for i in [2,4,5]:
    labels= map_clusters(chicago_problems,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_wo_sev1_"+label_types[i]+"_230608.html")